In [5]:
!date
!python --version

Thu Jun  6 02:27:27 AM UTC 2024
Python 3.10.12


# 極性推定（ネガポジ推定）の例
テキストとして書かれている文章に対して良い印象(positive)を持つか悪い印象(negative)を持つかを判定したい場面がある。代表的な算出方法としては、単語ごとに印象値を設定し、その平均値として文章の良し悪しを判断することが多い。

なお、分野により印象が変わることがあるため、一般的には手動チューニングしていくことが必要。例えば「音が小さい」という文は、洗濯機や冷蔵庫のような大型家電に対するレビュー文ならばポジティブな表現だろう。その一方でスピーカーやイヤホンに対するレビュー文ならば、（音が小さくて）聞き取りづらいというネガティブな表現と捉えることが妥当であるかもしれない。

ここでは代表的な例として[単語感情極性対応表](http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html)を用いて、出現した極性単語についての評価の平均値により文の極性を推定してみる。

- 極性辞書の例
    - [単語感情極性対応表](http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html), 2005年公開, 52,671語。Shift-JISで書かれてます。
    - [日本語評価極性辞書](https://www.cl.ecei.tohoku.ac.jp/Open_Resources-Japanese_Sentiment_Polarity_Dictionary.html), 2008年公開, 評価が客観的なものか主観的なものかに分けて評価. 評価値そのものはない。

In [6]:
# spacy, ginzaインストール
!pip install -U ginza ja_ginza

In [7]:
!curl -O https://ie.u-ryukyu.ac.jp/~tnal/2022/dm/static/r_assesment.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34834  100 34834    0     0  46268      0 --:--:-- --:--:-- --:--:-- 46321


In [8]:
import numpy as np
import pandas as pd
import spacy

nlp = spacy.load("ja_ginza")

assesment_df = pd.read_pickle('r_assesment.pkl')
assesment_df.head()

,title,grade,required,q_id,comment
0,工業数学Ⅰ,1,True,Q21 (1),特になし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。
3,工業数学Ⅰ,1,True,Q21 (2),特になし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。


## 極性辞書を扱いやすいように整形する
本の辞書は「単語:読み:品詞:評価値」の形式で行単位に極性単語が列挙されている。これを ``{単語:[品詞, 極性値]}`` の形で読み込もう。

In [9]:
# 極性辞書のダウンロード
!curl -O http://www.lr.pi.titech.ac.jp/~takamura/pubs/pn_ja.dic

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1683k  100 1683k    0     0  1030k      0  0:00:01  0:00:01 --:--:-- 1029k


In [10]:
pn_ja = {}
with open('pn_ja.dic', 'r', encoding='shift_jis') as f:
    for line in f.readlines():
        line = line.rstrip()
        items = line.split(':')
        word = items[0]
        pos = items[2]
        value = float(items[3])
        pn_ja[word] = [pos, value]

print(len(pn_ja))
print(pn_ja['優れる'])
print(pn_ja['死ぬ'])


52671
['動詞', 1.0]
['動詞', -0.999999]


## コメント文の極性を推定してみる

In [14]:
for comment in assesment_df['comment']:
    doc = nlp(comment)
    target_words = []
    target_evals = []
    for token in doc:
        if token.lemma_ in pn_ja.keys():
            target_words.append(token.lemma_)
            target_evals.append(pn_ja[token.lemma_][1])
    if len(target_words) > 0:
        score = np.mean(target_evals)
        print(f'comment = {comment}\n  total_score = {score:.5f}, words = {target_words}')

comment = 正直わかりずらい。むだに間があるし。
  total_score = -0.23687, words = ['正直', '間']
comment = 例題を取り入れて理解しやすくしてほしい。
  total_score = -0.20361, words = ['例題', '理解']
comment = スライドに書く文字をもう少しわかりやすくして欲しいです。
  total_score = -0.78033, words = ['スライド', '文字', 'もう', '少し', '欲しい']
comment = 話す際の間が多く聞き取りずらかった
  total_score = -0.73225, words = ['話す', '際', '間']
comment = 課題に関してですが、一度に3つの課題は大変なので１ずつでお願いします。
  total_score = -0.39952, words = ['課題', '関する', '一度', '課題', '大変', '願う']
comment = 高校の範囲では行列を扱わなかったので不安だったのですが、講義の中の説明で十分足りたことに安心しました。
  total_score = -0.42417, words = ['高校', '範囲', '行列', '扱う', 'ない', '不安', '講義', '中', '説明', '十分', '足りる', '安心']
comment = まだ１年次ということもあり、特に難しくなくてよかったです。
  total_score = -0.59872, words = ['年次', 'と', '難しい', 'ない']
comment = 機械トラブルの操作で授業が止まることがあった。
新しいことが学べて大学って感じがした。
  total_score = -0.47896, words = ['機械', 'トラブル', '操作', '授業', '新しい', '大学', '感じ']
comment = 線形代数は初めてだったけど、割と分かりやすい説明である程度理解できた。授業中にもっと演習（問題を出して時間をおいて回答させるなど）をもっと増やすといいと思う。
  total_score = -0.55665, words = ['線形', '代数', '初めて

In [12]:
# ん？最後の例は妙だな...

words = ['用いる', 'アプリケーション', 'チーム', '開発', 'とても', '経験', 'メンバー', 'や', '教える', '試す', 'と', '身']
for word in words:
    score = pn_ja[word][1]
    print(f'{word} = {score}')

用いる = -0.988007
アプリケーション = -0.145644
チーム = -0.492859
開発 = -0.255673
とても = -0.169067
経験 = -0.463699
メンバー = -0.337508
や = -0.278384
教える = -0.440103
試す = -0.502165
と = -0.215793
身 = -0.977377
